### Description: This is a sentimental analysis program that parses the tweets fetched from twitter using python

In [ ]:
#import the libraries
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")

In [ ]:
#Load the api_keys
keys = pd.read_csv("keys.csv")

In [ ]:
#Assign keys
consumerKey = keys.iloc[0, 0]
consumerSecret = keys.iloc[0, 1]
accessToken = keys.iloc[0, 2]
accessTokenSecret = keys.iloc[0, 3]
print(consumerKey)
print(consumerSecret)

In [ ]:
#Create the authentication object
authenticate = tweepy.OAuthHandler(consumerKey, consumerSecret)

#access token and access token secret
authenticate.set_access_token(accessToken, accessTokenSecret)

#create the api object while passing in the auth information
api = tweepy.API(authenticate, wait_on_rate_limit = True)

In [ ]:
#Extract 100 tweets from a twitter user
posts = api.user_timeline(screen_name = "BillGates", count = 100, lang = "en", tweet_mode = "extended")

#Print the last 5 tweets from the account
print("Show the 5 recent tweets: \n")

i = 1
for tweet in posts[0:5]:
    print(str(i) + ")" + tweet.full_text + "\n")
    i = i + 1

In [ ]:
#Create a dataframe with a column called Tweets
df = pd.DataFrame([tweet.full_text for tweet in posts], columns = ["Tweets"])
df.head()

In [ ]:
#clean the text

#Create a function to clean the tweets
def cleantxt(text):
    text = re.sub(r"@[A-Za-z0-9]+", "", text) #Removes @ mentions
    text = re.sub(r"#", "", text) #Removing the # symbol
    text = re.sub(r"RT[\s]+", "", text) #Removing RT
    text = re.sub(r"https?:\/\/\S+", "", text) #Removing hyperlinks
    return text

df["Tweets"] = df["Tweets"].apply(cleantxt)
    
#Show the cleaned text
df

In [ ]:
#Create a fuction to get the subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

#Create a fuction to get the subjectivity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

#Creating columns for subjectivity and polarity
df["Subjectivity"] = df["Tweets"].apply(getSubjectivity)
df["Polarity"] = df["Tweets"].apply(getPolarity)

#Show new columns 
df

In [ ]:
#Plot the wordcloud
allwords = "".join([twts for twts in df["Tweets"]])
wordCloud = WordCloud(width = 500, height = 300, random_state = 21, max_font_size = 119).generate(allwords)

plt.imshow(wordCloud, interpolation = "bilinear")
plt.axis("off")
plt.show()

In [ ]:
#Create a function to compute the negative, neutral and positive analysis
def getAnalysis(score):
    if score < 0:
        return "Negative"
    if score > 0:
        return "Positive"
    if score == 0:
        return "Neutral"

#Making column for analysis
df["Analysis"] = df["Polarity"].apply(getAnalysis)

#printing df with new analysis column
df

In [ ]:
#Printing all the positive tweets
j = 1
sortedDF = df.sort_values(by = ["Polarity"])
for i in range(0, sortedDF.shape[0]):
    if sortedDF["Analysis"][i] == "Positive":
        print(str(j) + ")" + sortedDF["Tweets"][i])
        print()
        j = j + 1

In [ ]:
#Print all the negative tweets
#Printing all the positive tweets
j = 1
sortedDF = df.sort_values(by = ["Polarity"])
for i in range(0, sortedDF.shape[0]):
    if sortedDF["Analysis"][i] == "Negative":
        print(str(j) + ")" + sortedDF["Tweets"][i])
        print()
        j = j + 1

In [ ]:
#Plot the polarity and subjectivity
plt.figure(figsize = (8, 6))
for i in range(0, df.shape[0]):
    plt.scatter(df["Polarity"][i], df["Subjectivity"][i], color = "Blue")
    
plt.title("Sentimental Analysis of Tweets")
plt.xlabel("Polarity")
plt.ylabel("Subjectivity")
plt.show()

In [ ]:
#Get the percentage of positive tweets
ptweets = df[df.Analysis == "Positive"]
ptweets = ptweets["Tweets"]

round(ptweets.shape[0] / df.shape[0] * 100, 1)

In [ ]:
#Get the percentage of negative tweets
ntweets = df[df.Analysis == "Negative"]
ntweets = ntweets["Tweets"]

round(ntweets.shape[0] / df.shape[0] * 100, 1)

In [ ]:
#Show the value counts
df["Analysis"].value_counts()

plt.title("Sentimental Analysis")
plt.xlabel("Sentiment")
plt.ylabel("Counts")
df["Analysis"].value_counts().plot(kind = "bar")
plt.show()